In [ ]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions

In [1]:
import pandas as pd

splits = {'train': 'train.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/BhabhaAI/DEITA-Complexity/" + splits["train"])

print(df.head)

/Users/thomazbonato/Desktop/Personal/Summer24/Coding/Azalea/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/thomazbonato/Desktop/Personal/Summer24/Coding/Azalea/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<bound method NDFrame.head of                                                   prompt  score
0                              Who is Jorge Luis Borges?      1
1      \nGiven a list of movies, determine which ones...      5
2      \nIdentify the neighboring major countries of ...      2
3                     What is salvation in christianity?      1
4                         What is the capital of France?      1
...                                                  ...    ...
65492  \nGiven a financial instrument known as the Se...      3
65493  \nConsidering various factors such as personal...      5
65494  \nIdentify the individual who pioneered the in...      3
65495  \nGiven a comprehensive happiness report that ...      5
65496  \nConsidering that a kids' soccer game typical...      5

[65497 rows x 2 columns]>


In [ ]:
def preprocessing(text):
